# Extraccion de datos

Importamos las librerias necesarias

In [5]:
import ast
import numpy as np
import pandas as pd
from tabulate import tabulate

Comenzamos con la extraccion de los datos proporcionados

In [2]:
# Importamos dataset australian_user_reviews y la guardamos en variable reviews
rows =[]

with open ('australian_user_reviews.json', encoding='MacRoman') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

reviews = pd.DataFrame(rows)

In [3]:
# Visualizamos el dataset de reviews
reviews.tail(3)

,user_id,user_url,reviews
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"[{'funny': '1 person found this review funny',..."


In [4]:
# Importamos el dataset australian_users_item y la guardamos en la variable items
rows =[]

with open ('australian_users_items.json', encoding='MacRoman') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

items = pd.DataFrame(rows)

In [5]:
# Visualizamos el dataset de items
items.head(3)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."


In [6]:
# Importamos el dataset output_steam_games y la guardamos en la variable games
games = pd.read_json('output_steam_games.json', lines=True)

In [7]:
# Visualizamos el dataset de juegos
games.head(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


# Limpieza de datos

A continuacion se procede con la normalizacion y limpieza de datos

In [8]:
# Convertimos los None a NaN para simplificar el proceso en el dataframe de games
games = games.replace([None], np.nan)

In [9]:
# Se eliminan las filas donde todas las columnas tengan NaN
games = games.dropna(how='all')

In [10]:
games.head(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com


In [11]:
# Desanidamos las listas de diccionarios de la columna reviews del dataset reviews
# Expandimos la columna "reviews" en filas separadas
df_reviews = reviews.explode('reviews', ignore_index=True)

# Convertimos los diccionarios en columnas separadas
df_reviews = pd.concat([df_reviews.drop(['reviews'], axis=1), df_reviews['reviews'].apply(pd.Series)], axis=1)

In [12]:
# Eliminamos la columna adicional y visualizamos si cargamos bien los datos
df_reviews = df_reviews.drop([0], axis=1)
df_reviews.head(3)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...


In [13]:
# Desanidamos las listas de diccionarios de la columna items del dataset items
data_desanidada = []

for index, row in items.iterrows():
    user_id = row['user_id']
    items_count = row['items_count']
    steam_id = row['steam_id']
    user_url = row['user_url']
    item = row['items']

    for i in item:
        new_row = {
        'user_id': user_id,
        'items_count': items_count,
        'steam_id' : steam_id,
        'user_url' : user_url,
        'item_id': i.get('item_id', ''),
        'item_name': i.get('item_name', ''),
        'playtime_forever': i.get('playtime_forever', ''),
        'playtime_2weeks': i.get('playtime_2weeks', '')
        }

        data_desanidada.append(new_row)

df_items = pd.DataFrame(data_desanidada)

In [14]:
df_items.head(3)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6,0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0,0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7,0


Miramos si hay valores nulos en los datasets, de ser asi se eliminan si se considera necesario

In [15]:
# Miramos cuantos datos nulos hay en cada columna de games
games.isnull().sum()

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64

In [16]:
# Se eliminan los registros que tienen el id como dato nulo ya que este es necesario para
# realizar join con los otros datasets
df_games = games.dropna(subset=['id'])

In [17]:
# Se eliminan los registros que tienen precio nulo para que al momento de realizar las
# operaciones no se vea afectado por estos datos
df_games = df_games.dropna(subset=['price'])

In [18]:
# Revisamos nuevamente la cantidad de nulos por columna para verificar que se eliminaros esos
# registros correctamente
df_games.isnull().sum()

publisher       7773
genres          3135
app_name           0
title           1932
url                0
release_date    1936
tags             130
reviews_url        0
specs            655
price              0
early_access       0
id                 0
developer       3156
dtype: int64

In [19]:
# Mirar datos nulos de cada columna de df_reviews
df_reviews.isnull().sum()

user_id         0
user_url        0
funny          28
posted         28
last_edited    28
item_id        28
helpful        28
recommend      28
review         28
dtype: int64

In [20]:
# Revisamos cuales son los registros de df_reviews que tienen datos nulos
df_reviews[df_reviews.isnull().any(axis=1)]

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
137,gdxsd,http://steamcommunity.com/id/gdxsd,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177,76561198094224872,http://steamcommunity.com/profiles/76561198094...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2559,76561198021575394,http://steamcommunity.com/profiles/76561198021...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10080,cmuir37,http://steamcommunity.com/id/cmuir37,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13767,Jaysteeny,http://steamcommunity.com/id/Jaysteeny,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15493,ML8989,http://steamcommunity.com/id/ML8989,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19184,76561198079215291,http://steamcommunity.com/profiles/76561198079...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20223,76561198079342142,http://steamcommunity.com/profiles/76561198079...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25056,76561198061996985,http://steamcommunity.com/profiles/76561198061...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26257,76561198108286351,http://steamcommunity.com/profiles/76561198108...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Despues de revisar los datos de cada columna se considera necesario eliminar los datos nulos de las columnas
# por lo que se procede a eliminar los datos nulos
df_reviews = df_reviews.dropna()

In [22]:
# Mirar datos nulos de df_reviews
df_reviews.isnull().sum()

user_id        0
user_url       0
funny          0
posted         0
last_edited    0
item_id        0
helpful        0
recommend      0
review         0
dtype: int64

In [23]:
# Mirar datos nulos de cada columna de df_items
df_items.isnull().sum()

user_id             0
items_count         0
steam_id            0
user_url            0
item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
dtype: int64

Al revisar los valores del dataset df_games mas especificamente en la columna price, se descubre que hay varios valores que indican que el juego es gratis o que su precio comienza en cierto valor. Por lo tanto se decide reemplazar todos los strings que haganreferencia a que el juego es gratis se reemplaza con 0 y las que dice que comienza en un valor se pondra este. Una vez hecho esto se convierte la columna price a float

In [24]:
# Se decide reemplazar los valores que digan free to play por 0 en la columna precio para poder
# cambiar el tipo de datos y asi realizar las operaciones dentro de las funciones
df_games['price'] = df_games['price'].replace('Free To Play', 0)
df_games['price'] = df_games['price'].replace('Free to Play', 0)
df_games['price'] = df_games['price'].replace('Free', 0)
df_games['price'] = df_games['price'].replace('Free Demo', 0)
df_games['price'] = df_games['price'].replace('Play for Free!', 0)
df_games['price'] = df_games['price'].replace('Install Now', 0)
df_games['price'] = df_games['price'].replace('Play WARMACHINE: Tactics Demo', 0)
df_games['price'] = df_games['price'].replace('Free Mod', 0)
df_games['price'] = df_games['price'].replace('Install Theme', 0)
df_games['price'] = df_games['price'].replace('Third-party', 0)
df_games['price'] = df_games['price'].replace('Play Now', 0)
df_games['price'] = df_games['price'].replace('Free HITMAN™ Holiday Pack', 0)
df_games['price'] = df_games['price'].replace('Play the Demo', 0)
df_games['price'] = df_games['price'].replace('Starting at $499.00', 499.00)
df_games['price'] = df_games['price'].replace('Starting at $449.00', 449.00)
df_games['price'] = df_games['price'].replace('Free to Try', 0)
df_games['price'] = df_games['price'].replace('Free Movie', 0)
df_games['price'] = df_games['price'].replace('Free to Use', 0)

In [25]:
# Se cambia el tipo de datos de la columna precio de string a float
df_games['price'] = df_games['price'].astype(float)

A continuacion se cambia los tipos de datos de las columnas que se consideran necesarias

In [26]:
# Se cambia el tipo de dato de la columa id del dataset de games para poder hacer los join
df_games['id'] = df_games['id'].astype(int)
df_games['id'] = df_games['id'].astype(str)

In [27]:
# Se cambia el tipo de dato de la columna genres del dataset games
df_games['genres'] = df_games['genres'].astype(str)

In [28]:
# Se reemplaza las palabras no validas por espacio en blanco
df_games['release_date'] = pd.to_datetime(df_games['release_date'].str.replace('SOON', '').str.replace('coming soon', '').str.replace('™', ''))

In [29]:
# Se convierte la fecha de release_date en el dataframe df_games
df_games['release_date'] = pd.to_datetime(df_games['release_date'])

In [30]:
# Se eliminan los espacios en blanco de id
df_games['id'] = df_games['id'].str.strip()

In [31]:
# Se eliminaran las fechas que estan incompletas (falta año) de la columna posted
# del dataset reviews para que no se presenten problemas al ejecutar las funciones
# para ello primero convertiremos estas fechas en nan
df_reviews['posted'] = df_reviews['posted'].replace(r'^Posted \w+ \d+\.$', pd.NaT, regex=True)
df_reviews = df_reviews.dropna(subset=['posted'])

In [32]:
# Se cambiara la columna posted del dataset reviews a formato datetime
# para que no haya problemas al ejecutar las funciones
df_reviews['posted'] = pd.to_datetime(df_reviews['posted'].str.replace('Posted ', ''), format='%B %d, %Y.')

In [33]:
# Eliminamos las columnas que no se van a utilizar del dataframe para acelerar
# el rendimiento y ahorrar memoria
columnas_games_drop = ['url','reviews_url']
df_games = df_games.drop(columnas_games_drop, axis = 1)

In [34]:
columnas_items_drop = ['steam_id', 'item_name', 'playtime_2weeks']
df_items = df_items.drop(columnas_items_drop, axis = 1)

In [35]:
columnas_reviews_drop = ['funny', 'last_edited', 'helpful']
df_reviews = df_reviews.drop(columnas_reviews_drop, axis = 1)

Se decide crear un dataframe nuevo llamado df_generos para no aumentar los registros de games y posteriormente poder acceder a ellos por medio de un join

In [36]:
# Se crea un dataframe de generos para que al momento de hacer las funciones se utilice este
df_generos = df_games[['id', 'genres']]

In [37]:
# Se eliminan los corchetes de los valores de la columna genres
df_generos['genres'] = df_generos['genres'].str.strip('[]')

C:\Users\Vaylen\AppData\Local\Temp\ipykernel_11444\950375968.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_generos['genres'] = df_generos['genres'].str.strip('[]')


In [38]:
# Se dividen las cadenas usando las comas (,) como delimitador
df_generos['genres'] = df_generos['genres'].str.split(', ')

C:\Users\Vaylen\AppData\Local\Temp\ipykernel_11444\2101267873.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_generos['genres'] = df_generos['genres'].str.split(', ')


In [39]:
# Se usa explode() para dividir la lista en múltiples filas
df_generos = df_generos.explode('genres')

In [40]:
# Se utiliza replace para eliminar las comillas
df_generos['genres'] = df_generos['genres'].str.replace("'", '')

In [41]:
df_generos.head(3)

,id,genres
88310,761140,Action
88310,761140,Casual
88310,761140,Indie


In [42]:
# Una vez creado este dataframe se elimina la columna genre del dataset,
# si se requiere en algun momento esta columna se utilizara df_generos
columnas_games_drop = ['genres']
df_games = df_games.drop(columnas_games_drop, axis = 1)

# Analisis de sentimiento

Para realizar el analisis de sentimiento primero se importaran las librerias necesarias para el preprocesamiento y creacion y entrenamiento del modelo de analisis de sentimiento

In [43]:
from textblob import TextBlob
from googletrans import Translator
from langdetect import detect
import string
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

Durante el proceso de ETL realizado anteriormente se noto algunos valores de reviews que se encontraban en un idioma diferente al ingles (idioma original del dataset), por lo que se procede a realizar las traducciones de esas reviews.

In [44]:
# Se realiza una funcion para traducir la columna de reviews si se ecuentra con un idioma diferente al ingles
def translate_review(text, target_language='en'):
    try:
        # Verificar si el texto tiene una longitud mínima antes de intentar la detección de idioma
        if len(text.strip()) < 3:  # Consideramos textos de menos de 3 caracteres como inválidos
            return text

        source_language = detect(text)
        if source_language == 'en':
            return text
        else:
            # Realizar la traducción solo si el idioma no es inglés
            translation = Translator.translate(text, dest=target_language)
            return translation.text

    except Exception as e:
        # Capturar cualquier excepción y devolver el texto original
        return text

In [45]:
# Se aplica la funcion anteriormente creada n la columna review
df_reviews['reviews_translated'] = df_reviews['review'].apply(translate_review)

Continuamos con el preprocesamiento de las reviews que ya fueron traducidas

In [46]:
# Se crea una funcion que se encarga de realizar el preprocesamiento básico de un 
# texto en inglés utilizando NLTK.
def preprocess_text(text):
    # Eliminación de stopwords
    stop_words = set(stopwords.words('english'))
    #tokens = [token for token in tokens if token not in stop_words]
    # Tokenización
    tokens = word_tokenize(text)
    # Eliminación de signos de puntuación
    tokens = [token for token in tokens if token not in string.punctuation]
    # Conversión a minúsculas
    tokens = [token.lower() for token in tokens]
    # Lematización
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Reconstrucción del texto preprocesado
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

In [47]:
# Se crea una funcion que se encarga de extraer las características del texto utilizando NLTK
# y devuelve un diccionario de características
def extract_features(text):
    features = {}
    words = word_tokenize(text)
    word_freq = FreqDist(words)
    for word, freq in word_freq.items():
        features[word] = freq
    return features

In [48]:
#Se convierte a tipo string la columna reviews_translated
df_reviews['reviews_translated'] = df_reviews['reviews_translated'].astype(str)

In [49]:
# Se aplica la funcion preprocess_text en reviews_translated
df_reviews['reviews_translated'] = df_reviews['reviews_translated'].apply(preprocess_text)

In [50]:
# Se aplica la funcion extract_features en reviews_translated
df_reviews['sentiment_score'] = df_reviews['reviews_translated'].apply(extract_features)

In [51]:
#Se convierte a tipo string la columna reviews_translated
df_reviews['reviews_translated'] = df_reviews['reviews_translated'].astype(str)

In [52]:
# Se crea una funcion que para cada review dada haga su analisis de sentimiento y la clasifique
# en positivo, negativo o neutro
def analyze_sentiment(review):
    blob = TextBlob(review)
    sentiment_score = blob.sentiment.polarity

    if sentiment_score > 0:
        sentiment = 2 #positivo
    elif sentiment_score < 0:
        sentiment = 0 #negativo
    else:
        sentiment = 1 #neutro
    
    return sentiment

In [53]:
# Aplicamos la funcion anteriormente creada en reviews_translated
df_reviews['sentiment'] = df_reviews['reviews_translated'].apply(analyze_sentiment)

In [54]:
# Se dividen los datos en train y test
X = df_reviews['reviews_translated']
y = df_reviews['sentiment']
X_train, X_test, y_train, y_test = train_test_split(df_reviews['reviews_translated'], df_reviews['sentiment'], test_size=0.2, random_state=42)

In [55]:
# Vectoriza tus datos de texto utilizando TF-IDF (Term Frequency-Inverse Document Frequency).
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Entrena el modelo de Regresión Logística.
classifier = LogisticRegression(max_iter=1000, multi_class='ovr', random_state=42)
classifier.fit(X_train, y_train)

# Realiza predicciones en el conjunto de prueba.
y_pred = classifier.predict(X_test)

# Evalúa el rendimiento del modelo.
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

In [57]:
columnas_review_drop = ['review', 'reviews_translated', 'sentiment_score']
df_reviews = df_reviews.drop(columnas_review_drop, axis = 1)

In [62]:
df_reviews.head(3)

,user_id,user_url,posted,item_id,recommend,sentiment
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-11-05,1250,True,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-07-15,22200,True,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-04-21,43110,True,2


# Convertir datasets en archivos csv


In [63]:
df_games.to_csv('games.csv', index=False)
df_items.to_csv('items.csv', index=False)
df_reviews.to_csv('reviews.csv', index=False)
df_generos.to_csv('generos.csv', index=False)

# Funciones para los endpoints

In [6]:
from pandas.io.parsers.readers import read_csv
import pandas as pd
df_games = read_csv('games.csv')
df_items = pd.read_parquet('items.parquet')
df_reviews = read_csv('reviews.csv')
df_generos = read_csv('generos.csv')

In [85]:
def userdata(user_id: str):
  if user_id in df_games['id'].values or user_id in df_items['user_id'].values or user_id in df_reviews['user_id'].values:

    # Gasto de usuario
    gasto = 0
    compras_usuario = pd.merge(df_games, df_items, left_on='id', right_on='item_id')
    usuario = compras_usuario[compras_usuario['user_id'] == user_id]['price']
    for i in usuario:
      gasto = gasto + i
    gasto = round(gasto, 2)
    gasto_usuario = f'El usuario {user_id} en total gasto ${gasto}'

    # Porcentaje de recomendacion
    recomendacion_usuario = df_reviews[df_reviews['user_id'] == user_id]['recommend']
    porcentaje = (recomendacion_usuario.sum()/len(recomendacion_usuario))*100
    porcentaje_usuario = f'El porcentaje de recomensacion de usuario {user_id} es del {porcentaje}%'

    # Cantidad de items
    cantidad_items = df_items[df_items['user_id'] == user_id]['items_count'].iloc[0]
    cantidad = f'La cantidad de items del usuario {user_id} es {cantidad_items}'

    return gasto_usuario, porcentaje_usuario, cantidad

  else:
    no_encontrado = f'El usuario {user_id} no fue encontrado en la base de datos'
    return no_encontrado

In [86]:
userdata('doctr')

C:\Users\Vaylen\AppData\Local\Temp\ipykernel_11444\2808988208.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if user_id in df_games['id'].values or user_id in df_items['user_id'].values or user_id in df_reviews['user_id'].values:


('El usuario doctr en total gasto $6760.32',
 'El porcentaje de recomensacion de usuario doctr es del 100.0%',
 'La cantidad de items del usuario doctr es 541')

In [87]:
def countreviews(fecha1:str, fecha2: str):
  # Cantidad de usuarios que realizaron reviews entre fecha1 y fecha2
  usuarios = df_reviews[(df_reviews['posted'] >= fecha1) & (df_reviews['posted'] <= fecha2)]['user_id']
  cantidad_usuarios = len(set(usuarios))
  cantidad = f'La cantidad de usuarios que realizaron reviews entre {fecha1} y {fecha2} son {cantidad_usuarios}'

  # Porcentaje de recomendacion
  recomendacion = df_reviews[(df_reviews['posted'] >= fecha1) & (df_reviews['posted'] <= fecha2)]['recommend']
  porcentaje = (recomendacion.sum()/len(recomendacion))*100
  porcentaje = round(porcentaje, 2)
  porcentaje_usuario = f'El porcentaje de recomensacion de los {cantidad_usuarios} es del {porcentaje}%'

  return cantidad, porcentaje_usuario

In [88]:
countreviews('2012-07-15', '2013-07-15')

('La cantidad de usuarios que realizaron reviews entre 2012-07-15 y 2013-07-15 son 1636',
 'El porcentaje de recomensacion de los 1636 es del 98.11%')

In [33]:
def genero(genero: str):
    # Combinar DataFrames
    generos = pd.merge(df_generos, df_items, left_on='id', right_on='item_id', how='inner')

    # Filtrar por género
    genero_data = generos[generos['genres'] == genero]

    if genero_data.empty:
        return f'El género {genero} no fue encontrado en la base de datos'

    # Calcular el ranking del género
    tiempo_juego = genero_data.groupby('genres')['playtime_forever'].sum()
    tiempo_juego = tiempo_juego.reset_index()
    tiempo_juego = tiempo_juego.sort_values(by='playtime_forever', ascending=False)
    tiempo_juego['ranking'] = tiempo_juego.index + 1

    lugar_ranking = tiempo_juego.loc[tiempo_juego['genres'] == genero, 'ranking'].values[0]
    ranking = f'El género {genero} está ubicado en el puesto {lugar_ranking} del ranking'
    
    return ranking

In [37]:
genero('Racing')

'El género Racing está ubicado en el puesto 1 del ranking'

In [21]:
def userforgenre(genero: str):
    # Combinar DataFrames
    generos_usuario = pd.merge(df_generos, df_items, left_on='id', right_on='item_id', how='inner')

    # Filtrar por género
    generos_usuario = generos_usuario[generos_usuario['genres'] == genero]

    if generos_usuario.empty:
        return f'El género {genero} no fue encontrado en la base de datos'

    # Calcular las 5 mejores tuplas
    generos_usuario['playtime_forever'] = generos_usuario['playtime_forever'] / 60
    top_usuarios = generos_usuario.nlargest(5, 'playtime_forever')[['user_id', 'user_url']]
    top_usuarios['rank'] = top_usuarios.reset_index().index + 1

    tuplas = top_usuarios[['rank', 'user_id', 'user_url']].to_records(index=False).tolist()

    return tuplas

In [24]:
userforgenre('Action')

[(1, 'Evilutional', 'http://steamcommunity.com/id/Evilutional'),
 (2, 'shinomegami', 'http://steamcommunity.com/id/shinomegami'),
 (3,
  '76561197977470391',
  'http://steamcommunity.com/profiles/76561197977470391'),
 (4, 'theoriginalmasta', 'http://steamcommunity.com/id/theoriginalmasta'),
 (5,
  '76561198027114566',
  'http://steamcommunity.com/profiles/76561198027114566')]

In [13]:
def desarrollador(desarrollador: str):
    # Filtrar juegos del desarrollador
    games_items = pd.merge(df_games, df_items, left_on='id', right_on='item_id')
    desarrollador_data = games_items[games_items['developer'] == desarrollador]

    if desarrollador_data.empty:
        return f'El desarrollador {desarrollador} no fue encontrado en la base de datos'

    # Cantidad de items
    cantidad_items_desarrollador = len(desarrollador_data)

    # Porcentaje de contenido gratuito por empresa desarrolladora y año
    contenido = desarrollador_data[['id', 'developer', 'price', 'release_date']]
    contenido = contenido.dropna(subset=['release_date'])
    contenido['year'] = contenido['release_date'].str.split('-').str[0].astype(int)
    contenido_gratis = contenido[contenido['price'] == 0]
    
    desarrollador_year_gratis = contenido_gratis.groupby(['developer', 'year'])[['id']].nunique().reset_index()
    desarrollador_year = contenido.groupby(['developer', 'year'])[['id']].nunique().reset_index()
    
    resultado = pd.merge(desarrollador_year_gratis, desarrollador_year, on=['developer', 'year'], how='inner')
    resultado['porcentaje'] = (resultado['id_x'] / resultado['id_y'] * 100).astype(int)
    
    # Formatear resultados
    cantidad = f'La cantidad de items del desarrollador {desarrollador} es {cantidad_items_desarrollador}'
    desarrollador_porcentaje = tabulate(resultado, headers='keys', tablefmt='psql')
    
    return cantidad, desarrollador_porcentaje

In [16]:
desarrollador('Ankama Studio')

('La cantidad de items del desarrollador Ankama Studio es 2758',
 '+----+---------------+--------+--------+--------+--------------+\n|    | developer     |   year |   id_x |   id_y |   porcentaje |\n|----+---------------+--------+--------+--------+--------------|\n|  0 | Ankama Studio |   2014 |      1 |      1 |          100 |\n|  1 | Ankama Studio |   2015 |      1 |      1 |          100 |\n+----+---------------+--------+--------+--------+--------------+')

In [96]:
def sentiment_analysis(year: int):
    games_reviews = pd.merge(df_games, df_reviews, left_on='id', right_on='item_id')
    sentimiento = games_reviews[['release_date', 'sentiment']]
    sentimiento = sentimiento[sentimiento['release_date'].notna()]
    sentimiento['year'] = sentimiento['release_date'].str.split('-').str[0].astype(int)
    
    if year in sentimiento['year'].values:
        filtro_year = sentimiento[sentimiento['year'] == year]
        sentiment_mapping = {2: 'positivo', 1: 'neutro', 0: 'negativo'}
        filtro_year['sentiment'] = filtro_year['sentiment'].map(sentiment_mapping)
        sentiment_counts = filtro_year['sentiment'].value_counts().to_dict()
        
        return sentiment_counts
    
    else:
        no_encontrado = f'El año {year} no fue encontrado en la base de datos'
        return no_encontrado

In [97]:
sentiment_analysis(2015)

C:\Users\Vaylen\AppData\Local\Temp\ipykernel_11444\28946878.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtro_year['sentiment'] = filtro_year['sentiment'].map(sentiment_mapping)


{'positivo': 2497, 'negativo': 1126, 'neutro': 1004}